In [3]:
from abc_utils import *
import pandas as pd
from hmmlearn import hmm
import numpy as np
from sklearn.metrics import accuracy_score

In [4]:
train_set, train_lengths, train_indicies, val_set, val_lengths, val_indicies = load_datasets()

Demonstration of using OG_Dataset()

In [22]:
# Instantiate the object
og_dataset = OG_Dataset()

# Filter the training set to only contain songs in 'major' keys
major_train_df, major_train_lengths = og_dataset.filter_df(train_set, train_lengths, col_to_filter='keys', filter_str='major', split_type='train')

# Filter the validation set to only contain songs in 'major' keys
major_val_df, major_val_lengths  = og_dataset.filter_df(train_set, train_lengths, col_to_filter='keys', filter_str='major', split_type = 'val')

### Using these Data subsets (This is basically identical to pomegrante_test_Adam.ipynb)

In [13]:
def ffill_obs(melody_obs: np.ndarray, unique_obs: dict) -> np.ndarray:
    # make a smaller array out of the unique observations
    possible_obs = list(set(unique_obs.flatten()))

    df_melody_obs = pd.Series(melody_obs)
    df_melody_obs[~df_melody_obs.isin(possible_obs)] = np.nan

    # fill forward first to fill all the holes
    df_melody_obs.ffill(inplace=True)

    # then fill backward to catch the case where
    # the beginning is empty
    df_melody_obs.bfill(inplace=True)

    return df_melody_obs.values.flatten()

In [14]:
def chord_accuracy(full_pred: np.array, true_states: np.array, num_chords: int=None, num_notes: int=None):
    '''
    Given the predicted matrix of states, compute the misclassification rate compared with the true_observations.
    Could be edited in the future to also compute the accuracy of our predicted note sequence.
    '''
    # check to make sure these are specified correctly
    if num_chords is None:
        raise ValueError("num_chords must be specified")
    if num_notes is None:
        raise ValueError("num_notes must be specified")
    
    # obtain the actual predicted chords 
    pred_chords = full_pred[:, num_chords-1]
    true_chords = true_states[:len(pred_chords), num_chords-1]

    # obtain the accuracy
    chord_acc = accuracy_score(true_chords, pred_chords)
    
    return chord_acc

In [15]:
def fit_model(train_set: pd.DataFrame, train_lengths: pd.Series, num_chords: int=1, num_notes: int=0, subset: bool=False, indices=None, lam: int=None, trans_prior: float=0.1, emissions_prior: float=0.1):
    """ 
    Takes in the train set and parameters for the state space and returns the trained model, along with all of the dictionaries needed to decode the model as a tuple.

    To train on a smaller subset of the full train set, use the subset argument and pass in the indices needed. Uses the load_song_subset function.
    """
    # check if we want to do a subset of the full train set; if so, perform it
    if subset:
        # check that indices are specified; raise and error if not
        if indices is None:
            raise ValueError("Indices must be specified if subset=True")
        train_set, _ = load_song_subset(train_set, train_lengths, indices)

    # obtain the states and observations from the songs
    true_states, true_observations = dataframe_to_states(train_set, num_chords, num_notes)
    
    # create the transition matrices for the model
    transition_matrix, emission_probs, unique_states, unique_obs, states_to_index, observation_to_index = states_to_transition(true_states, true_observations, lam, trans_prior, emissions_prior)

    # now initialize the model and set the matrices for it
    model = hmm.CategoricalHMM(n_components=transition_matrix.shape[0], init_params='')
    model.transmat_ = transition_matrix.T
    model.emissionprob_ = emission_probs.T

    # starting_state = np.zeros(unique_states.shape[1])
    # starting_state_index = states_to_index[tuple(starting_state)]

    # start_probs = np.zeros(transition_matrix.shape[0])
    # start_probs[starting_state_index] = 1
    
    model.startprob_ = np.ones(transition_matrix.shape[0]) / transition_matrix.shape[0]

    # return the model,  the dictionaries
    return model, (unique_states, unique_obs, states_to_index, observation_to_index)

In [16]:
def predict_states(model: hmm.CategoricalHMM, all_dicts: tuple, observation: np.ndarray, song_lengths: list):
    """
    Uses the model to decode an observation. The all_dicts tuple should contain the model dictionaries returned from fit_model
    Returns the predicted states.
    """
    # unpack the tuple to get what we need
    unique_states, unique_obs, _, observation_to_index = all_dicts

    # perform a forward fill on the observation in case there are any values in it that we have never seen before
    observation = ffill_obs(observation, unique_obs)
    
    # get the indices of the observation
    observation_indices = np.array([int(observation_to_index[(o,)]) for o in observation])

    # get the predicted state indices
    _, pred_indices = model.decode(observation_indices.reshape(-1, 1), lengths=song_lengths)

    # use the unique_states dictionary to take the indices to the actual states
    pred_states = unique_states[pred_indices, :]

    # return the predicted states
    return pred_states

In [17]:
def redact(seq, lam):
    """
    Redact a sequence of chords to only contain lam repetitions of any given chord in the sequence. Takes in a one-dimensional sequence and returns the 
    shortened sequence
    """
    if len(seq.shape) != 1:
        raise TypeError("array must be 1-dimensional")

    # start building a mask for the sequence: True if we are below lam repetitions, False otherwise
    curr_val = seq[0]
    length = 1
    mask = [True]

    # iterate through and create the mask
    for i in range(1, len(seq)):
        if seq[i] == curr_val:
            if length <= lam - 1:
                mask.append(True)
            else:
                mask.append(False)
            length += 1
        else:
            mask.append(True)
            curr_val = seq[i]
            length = 1

    # mask out the values and return
    mask = np.array(mask)
    return seq[mask]

In [18]:
def get_prediction(model, all_dicts, val_set: pd.DataFrame, val_lengths: pd.Series, num_chords: int=1, num_notes: int=0, subset: bool=False, indices=None, do_print: bool=True):
    if subset:
        val_set, val_lengths = load_song_subset(val_set, val_lengths, indices)
    
    true_states, new_song_obs = dataframe_to_states(val_set, num_chords, num_notes)

    # get the predicted states (chop off the first element of the songs because it added a 0)
    pred_states = predict_states(model, all_dicts, new_song_obs[1:], val_lengths.values.flatten().tolist())

    
    # print the results, then return the results and the accuracy
    if do_print:
        print("Pred\t\tTrue")
        cumul = np.cumsum(val_lengths.values)
        for i in range(len(pred_states)):
            if i in set(cumul):
                print("----- New Song -----")
            print(f"{pred_states[i]}\t\t{true_states[i]}")

    # get the accuracy
    accuracy = chord_accuracy(pred_states, true_states, num_chords, num_notes)
    print("Accuracy:", accuracy)

    return pred_states, accuracy

In [19]:
# fit the model
num_chords = 1
num_notes = 0

# run through the lambda values and get the accuracy on the validation set for that lambda value
for lam in [None, 4, 3, 2]:
    model, all_dicts = fit_model(major_train_df, major_train_lengths, num_chords, num_notes, lam=lam, trans_prior=0, emissions_prior=0)

    # get the prediction
    print(fr"$\lambda$ = {lam}")
    pred_states, acc = get_prediction(model, all_dicts, major_val_df, major_val_lengths, do_print=False)

Processing states: 100%|██████████| 1931101/1931101 [02:11<00:00, 14731.42it/s]


$\lambda$ = None


Processing states: 100%|██████████| 495132/495132 [00:38<00:00, 12817.78it/s]


Accuracy: 0.11363232430947706


Processing states: 100%|██████████| 1931101/1931101 [02:16<00:00, 14139.58it/s]


...

Accuracy: 0.2681163810862558


Processing states: 100%|██████████| 1931101/1931101 [01:56<00:00, 16611.07it/s]


$\lambda$ = 2


Processing states: 100%|██████████| 495132/495132 [00:29<00:00, 17000.29it/s]


Accuracy: 0.2933258201853243


In [20]:
# get the true states (should be the same no matter what)
num_chords = 1
num_notes = 0
true_states, _ = dataframe_to_states(major_val_df, num_chords, num_notes)

# get the accuracy of the all I sequence
baseline = accuracy_score(true_states[:, num_chords-1].flatten(), np.ones(len(true_states)))
print("Baseline Accuracy:", baseline)

Processing states: 100%|██████████| 495132/495132 [00:28<00:00, 17140.27it/s]

Baseline Accuracy: 0.3368670640009856
